In [1]:
import pickle
import numpy as np
from scipy import optimize

In [20]:
method = "rpb"
name_data="mnist"
model="fcn"
layers=4
objective="fclassic"
T=6
split="geometric"
gamma_t=0.5
recursive_step_1=False
T_splits = [1875, 1875, 3750, 7500, 15000, 30000]
#T_splits = [7500, 7500, 15000, 30000]
#T_splits = [30000,30000]
T = len(T_splits)

In [21]:
#seeds = np.arange(5)
seeds = [0]

In [22]:
risks = []
train_loss = []
test_loss = []
for seed in seeds:
    if method == "rpb":
        exp_settings = f"{name_data}_{model}_{layers}_{objective}_{split}_{T}_{recursive_step_1}_{gamma_t}_{seed}.pt"
    else:
        exp_settings = f"{name_data}_{model}_{layers}_{objective}_{split}_{seed}.pt"
    results_dir = f"./results/{method}/results_" + exp_settings

    with open(results_dir, "rb") as handle:
        result_seed = pickle.load(handle)

    if method == "Recursive-T":
        risks.append(result_seed["risk"][-1])
        train_loss.append(result_seed["train_loss"][-1])
        test_loss.append(result_seed["test_loss"][-1])
    elif method == "InformedExcess":
        risks.append(result_seed["risk"][0])
        train_loss.append(result_seed["train_loss"])
        test_loss.append(result_seed["test_loss"])
    else:
        risks.append(result_seed["risk"])
        train_loss.append(result_seed["train_loss"])
        test_loss.append(result_seed["test_loss"])

In [199]:
print(np.mean(risks))
print(np.std(risks))
risks

0.19741307288013943
0.0033783960836538856


[0.20299642777491905,
 0.19566946167410015,
 0.19869031685484906,
 0.19283831579347072,
 0.19687084230335813]

In [200]:
print(np.mean(train_loss))
print(np.std(train_loss))
train_loss

0.11054333333333335
0.003165834978502687


[0.11161666666666667,
 0.10875,
 0.10566666666666667,
 0.11511666666666667,
 0.11156666666666666]

In [201]:
print(np.mean(test_loss))
print(np.std(test_loss))
test_loss

0.09214
0.0013440238093129157


[0.0928, 0.0908, 0.0927, 0.0904, 0.094]

In [23]:
result_seed

{'loss': [0.3,
  array([0.8020129 , 0.14513548, 0.03836559], dtype=float32),
  array([0.93678224, 0.12458666, 0.0408    ], dtype=float32),
  array([0.9446476 , 0.10830476, 0.03548571], dtype=float32),
  array([0.96      , 0.10428889, 0.03495555], dtype=float32),
  array([0.9637667 , 0.09813333, 0.03116667], dtype=float32)],
 'kl': [array(1457.8726, dtype=float32),
  array(2808.4126, dtype=float32),
  array(535.4685, dtype=float32),
  array(220.26709, dtype=float32),
  array(115.32048, dtype=float32),
  array(84.28107, dtype=float32)],
 'excess_risk': [0.16308676228490693,
  0.11427749912006058,
  0.08604685879555296,
  0.08279835605349539,
  0.08163027525829492],
 'risk': [0.41385722688535564,
  0.37001537572758475,
  0.299285186983853,
  0.23568945228747945,
  0.20064308219723512,
  0.1819518163569125],
 'train_loss': 0.09815,
 'test_loss': [0.2915, 0.1405, 0.1188, 0.1012, 0.0979, 0.0937]}

In [88]:
exc = result_seed["loss"][1]
-0.5 + 0.5 * exc[0] + 0.5 * exc[1] + 0.5 * exc[2]

-0.021462365984916687

In [65]:

def KL(Q, P):
    """
    Compute Kullback-Leibler (KL) divergence between distributions Q and P.
    """
    return sum([q * np.log(q / p) if q > 0.0 else 0.0 for q, p in zip(Q, P)])


def KL_binomial(q, p):
    """
    Compute the KL-divergence between two Bernoulli distributions of probability
    of success q and p. That is, Q=(q,1-q), P=(p,1-p).
    """
    return KL([q, 1.0 - q], [p, 1.0 - p])


def get_binominal_inv(n, k, delta):
    for p in np.linspace(1, 0, 100001):
        if binom.pmf(k, n, p) >= delta:
            return p


def solve_kl_sup(q, right_hand_side):
    """
    find x such that:
        kl( q || x ) = right_hand_side
        x > q
    """
    f = lambda x: KL_binomial(q, x) - right_hand_side

    if f(1.0 - 1e-9) <= 0.0:
        return 1.0 - 1e-9
    else:
        return optimize.brentq(f, q, 1.0 - 1e-11)

In [173]:
""" Stats of Uninformed """
n_bound = 60000
mc_samples = 60000
delta_test=0.01
delta=0.025
method = "Uninformed"
name_data = "fmnist"
model = "cnn"
objective = "fclassic"
seeds = np.arange(5)

kln = []
risks = []
train_loss = []
eval_loss = []
test_loss = []
for seed in seeds:
    exp_settings = f"{name_data}_{model}_{objective}_{seed}.pt"
    results_dir = f"./results/{method}/results_" + exp_settings

    with open(results_dir, "rb") as handle:
        result_seed = pickle.load(handle)
    ## Basic
    risks.append(result_seed["risk"])
    train_loss.append(result_seed["train_loss"])
    eval_loss.append(result_seed["train_loss"]) # should be eval_loss
    test_loss.append(result_seed["test_loss"])

    kln.append(result_seed["kl"]/n_bound)

# train loss
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
# for bound
print("eval_loss: ", np.mean(eval_loss), " ; ", np.std(eval_loss))
print("kln: ", np.mean(kln)," ; ", np.std(kln))
print("risks: ", np.mean(risks), " ; ", np.std(risks))
# test loss
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))

train_loss:  0.36851  ;  0.0037168475292316708
eval_loss:  0.36851  ;  0.0037168475292316708
kln:  0.01543332255045573  ;  0.00013166799888221064
risks:  0.4619243265818195  ;  0.003768512132500606
test_loss:  0.37232  ;  0.005380483249671908


In [168]:
""" Stats of Informed """
n_bound = 30000
mc_samples = 30000
delta_test=0.01
delta=0.025
method = "Informed"
name_data = "mnist"
model = "fcn"
objective = "fclassic"
seeds = np.arange(5)

kln = []
risks = []
train_loss = []
eval_loss = []
test_loss = []
for seed in seeds:
    exp_settings = f"{name_data}_{model}_{objective}_{seed}.pt"
    results_dir = f"./results/{method}/results_" + exp_settings

    with open(results_dir, "rb") as handle:
        result_seed = pickle.load(handle)
    
    print("result_seed: ", seed, result_seed)
    ## Basic
    risks.append(result_seed["risk"])
    train_loss.append(result_seed["train_loss"])
    eval_loss.append(result_seed["eval_loss"])
    test_loss.append(result_seed["test_loss"])
    kln.append(result_seed["kl"]/n_bound)

# Train loss
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
# For bound
print("eval_loss: ", np.mean(eval_loss), " ; ", np.std(eval_loss))
print("kln: ", np.mean(kln)," ; ", np.std(kln))
print("risks: ", np.mean(risks), " ; ", np.std(risks))
# Test loss
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))

result_seed:  0 {'kl': array(71.11655, dtype=float32), 'risk': 0.37849925130634987, 'train_loss': 0.33725, 'eval_loss': 0.33486666666666665, 'test_loss': 0.3362}
result_seed:  1 {'kl': array(75.47776, dtype=float32), 'risk': 0.3834928933438333, 'train_loss': 0.33876666666666666, 'eval_loss': 0.3388, 'test_loss': 0.324}
result_seed:  2 {'kl': array(71.56778, dtype=float32), 'risk': 0.3750388205977402, 'train_loss': 0.33898333333333336, 'eval_loss': 0.3314, 'test_loss': 0.3358}
result_seed:  3 {'kl': array(73.19232, dtype=float32), 'risk': 0.37799861320456957, 'train_loss': 0.3353833333333333, 'eval_loss': 0.33393333333333336, 'test_loss': 0.3307}
result_seed:  4 {'kl': array(71.14758, dtype=float32), 'risk': 0.3797721538435447, 'train_loss': 0.3419333333333333, 'eval_loss': 0.3361, 'test_loss': 0.3353}
train_loss:  0.33846333333333334  ;  0.0021615683606533895
eval_loss:  0.33502  ;  0.0024398178438381686
kln:  0.0024166799418131506  ;  5.5694354537530375e-05
risks:  0.37896034645920756

In [176]:
""" Stats of InformedExcess """
n_bound = 30000
mc_samples = 30000
delta_test=0.01
delta=0.025
method = "InformedExcess"
name_data = "fmnist"
model = "cnn"
objective = "fclassic"
seeds = np.arange(5)
# Excess loss
rv = np.array([-1, 0, 1])
js = rv[1:]
js_minus = rv[1:] - rv[0:-1]

kln = [] # kl/n
train_loss = []
eval_loss = []
excess_loss = []
h_loss = []
test_loss = []
risks = [] # the risk
excess_risks = [] # the excess risk
h_risks = [] # reference classifier h risks

for seed in seeds:
    exp_settings = f"{name_data}_{model}_{objective}_{seed}.pt"
    results_dir = f"./results/{method}/results_" + exp_settings

    with open(results_dir, "rb") as handle:
        result_seed = pickle.load(handle)
    print("result_seed: ", seed, result_seed)

    ## Basic
    risks.append(result_seed["risk"][0])
    h_risks.append(result_seed["risk"][1])
    excess_risks.append(result_seed["risk"][0] - result_seed["risk"][1])
    train_loss.append(result_seed["train_loss"])
    test_loss.append(result_seed["test_loss"])

    # excess risk
    excess_loss_1_seed = result_seed["risk"][2][0]
    excess_loss_2_seed = result_seed["risk"][2][1]
    excess_loss_seed = rv[0] + js_minus[0] * excess_loss_1_seed + js_minus[1] * excess_loss_2_seed
    excess_loss.append(excess_loss_seed)
    kl_seed = result_seed["kl"] ; kln.append(kl_seed/n_bound)

    # h risk
    h_loss_seed = result_seed["risk"][3] ; h_loss.append(h_loss_seed)

# Train loss
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
# For bound
## excess bound
print("kln: ", np.mean(kln)," ; ", np.std(kln))
print("excess_loss: ", np.mean(excess_loss), " ; ", np.std(excess_loss))
print("excess_risk: ", np.mean(excess_risks)," ; ", np.std(excess_risks))
## h bound
print("h_loss: ", np.mean(h_loss), " ; ", np.std(h_loss))
print("h_risks: ", np.mean(h_risks), " ; ", np.std(h_risks))
## bound
print("risks: ", np.mean(risks), " ; ", np.std(risks))
# Test loss
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))

result_seed:  0 {'kl': array(865.04755, dtype=float32), 'risk': (0.42857907672185014, 0.08991579254220548, array([0.97653335, 0.2221    ], dtype=float32), 0.08506666666666667), 'train_loss': 0.2833833333333333, 'test_loss': 0.2876}
result_seed:  1 {'kl': array(879.98584, dtype=float32), 'risk': (0.41770111109182373, 0.0890955523978246, array([0.97443336, 0.21273333], dtype=float32), 0.08426666666666667), 'train_loss': 0.27216666666666667, 'test_loss': 0.2824}
result_seed:  2 {'kl': array(876.2779, dtype=float32), 'risk': (0.4292791967207983, 0.09186336119398031, array([0.97466666, 0.2207    ], dtype=float32), 0.08696666666666666), 'train_loss': 0.28246666666666664, 'test_loss': 0.2895}
result_seed:  3 {'kl': array(892.44244, dtype=float32), 'risk': (0.4327480367073655, 0.09193168430010411, array([0.9729667, 0.2231   ], dtype=float32), 0.08703333333333334), 'train_loss': 0.2843333333333333, 'test_loss': 0.2887}
result_seed:  4 {'kl': array(901.6829, dtype=float32), 'risk': (0.4323154928

In [82]:
""" Stats of Recursive-T """
T_splits = [7500, 7500, 15000, 30000]#[1875, 1875, 3750, 7500, 15000, 30000]
T = len(T_splits)
n_total = 60000
n_bound = [n_total_i - T_splits_i for n_total_i , T_splits_i in zip([60000]*T,T_splits)]# ; print("n_bound", n_bound)
mc_samples = n_bound
delta_test=0.01
delta=0.025
method = "Recursive-T"
name_data = "fmnist"
model = "cnn"
objective = "fclassic"
seeds = np.arange(5)
# Excess loss
gamma_t = 0.5
rv = np.array([-gamma_t, 0, 1-gamma_t, 1])
js = rv[1:]# ; print("js", js)
js_minus = rv[1:] - rv[0:-1]# ; print("js_minus", js_minus)


# B1
B_1_loss = []
B_1_kln = []
B_1_risk = []

# Et
Et_loss = []
Et_kln = []
Et_risk = []

kln = [] # kl/n
train_loss = []
eval_loss = []
excess_loss = []
h_loss = []
test_loss = []
risks = [] # the risk
excess_risks = [] # the excess risk
h_risks = [] # reference classifier h risks

for seed in seeds:
    exp_settings = f"{name_data}_{model}_{objective}_{T}_{seed}.pt"
    results_dir = f"./results/{method}/results_" + exp_settings

    with open(results_dir, "rb") as handle:
        result_seed = pickle.load(handle)
    print("result_seed: ", seed, result_seed)

    # Basic for last posterior
    risks.append(result_seed["risk"][-1])
    train_loss.append(result_seed["train_loss"][-1])
    test_loss.append(result_seed["test_loss"][-1])

    # More statistics
    ## B_1
    B_1_loss_seed = result_seed["loss"][0]# ; print("B_1_loss_seed", B_1_loss_seed)
    B_1_loss.append(B_1_loss_seed)
    B_1_kl_seed = result_seed["kl"][0]# ; print("B_1_kl_seed", B_1_kl_seed)
    B_1_kln.append(B_1_kl_seed/n_total)
    B_1_risk.append(result_seed["risk"][0])
    #B_1_risk_seed = solve_kl_sup(B_1_loss_seed, np.log(T / delta_test) / n_total)# ; print("B_1_risk_seed", B_1_risk_seed)
    #B_1_risk_seed = solve_kl_sup(B_1_risk_seed, (B_1_kl_seed + np.log((2 * T * np.sqrt(n_total)) / delta)) / n_total)
    #print("B_1_risk_seed ", B_1_risk_seed, " risk recorded ", result_seed["risk"][0])
    #Excess_Et_seed = result_seed["loss"][1:] ; print("Excess_Et_seed", Excess_Et_seed)
    

    ## excess risk
    Et_loss_seed = result_seed["loss"][1:] ; print("Et_loss_seed", Et_loss_seed)
    for t in range(T):
        
    #excess_loss_T_seed = result_seed["risk"][1:]
    #excess_loss_2_seed = result_seed["risk"][2][1]
    #excess_loss_seed = rv[0] + js_minus[0] * excess_loss_1_seed + js_minus[1] * excess_loss_2_seed
    #excess_loss.append(excess_loss_seed)
    #kl_seed = result_seed["kl"] ; kln.append(kl_seed/n_bound)

    # h risk
    #h_loss_seed = result_seed["risk"][3] ; h_loss.append(h_loss_seed)

# Train loss
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
# For bound
## B_1
print("B_1 loss: ", np.mean(B_1_loss), " ; ", np.std(B_1_loss))
print("B_1_kln: ", np.mean(B_1_kln), " ; ", np.std(B_1_kln))
## excess bound
#print("kln: ", np.mean(kln)," ; ", np.std(kln))
#print("excess_loss: ", np.mean(excess_loss), " ; ", np.std(excess_loss))
#print("excess_risk: ", np.mean(excess_risks)," ; ", np.std(excess_risks))
## h bound
#print("h_loss: ", np.mean(h_loss), " ; ", np.std(h_loss))
#print("h_risks: ", np.mean(h_risks), " ; ", np.std(h_risks))
## bound
print("risks: ", np.mean(risks), " ; ", np.std(risks))
# Test loss
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))

result_seed:  0 {'loss': [0.24588333333333334, array([0.8829905 , 0.06594285, 0.06594285], dtype=float32), array([0.93648887, 0.05351111, 0.05351111], dtype=float32), array([0.9436, 0.0568, 0.0568], dtype=float32)], 'kl': [array(2982.1985, dtype=float32), array(386.2584, dtype=float32), array(85.042496, dtype=float32), array(51.494453, dtype=float32)], 'excess_loss': [0.06745334041530293, 0.051698095120574106, 0.05970857691955134], 'risk': [0.40227381030633697, 0.26859024556847144, 0.18599321790480983, 0.15270518587195625], 'train_loss': [0.2753333333333333, 0.19135, 0.18353333333333333, 0.17743333333333333], 'test_loss': [0.2479, 0.1955, 0.1777, 0.2043]}
Et_loss_seed 3 [array([0.8829905 , 0.06594285, 0.06594285], dtype=float32), array([0.93648887, 0.05351111, 0.05351111], dtype=float32), array([0.9436, 0.0568, 0.0568], dtype=float32)]
result_seed:  1 {'loss': [0.2509166666666667, array([0.88405716, 0.06643809, 0.06643809], dtype=float32), array([0.9384222, 0.0614   , 0.0614   ], dtype